# S3W8D4: LangChain 入门 (The Orchestrator)

前三天我们像是在“手搓代码”：手动发 HTTP 请求、手动拼 Prompt 字符串、手动解析 JSON。虽然这样能让你理解底层原理，但在大型项目（几十个步骤的 Agent）中，这种写法会像面条一样混乱。

今天，我们要引入 **AI 应用开发的“操作系统”——LangChain**。

LangChain 最迷人的地方在于 **LCEL (LangChain Expression Language)**，它让写 AI 逻辑变得像写 Linux 管道命令（Pipe）一样优雅。

**🎯 今日目标**

1. **环境配置**：用 `LangChain` 连接硅基流动 (SiliconFlow) 的 DeepSeek 模型。
2. **核心概念**：掌握 `ChatPromptTemplate` 和 `LCEL` 管道写法 (`|`)。
3. **重构实战**：用 3 行代码重写前 3 天写了 50 行的“法律意图分类器”。

## 📖 1 理论知识讲解 (Theory & Analogy)

**为什么要用 LangChain？**

> **🧬 医学图像类比：手动处理 vs. Nipype 流水线**
> * **Day 1-3 的写法**：就像你用 Python 手写脚本处理医学图像。
> * `img = load_dicom(...)`
> * `mask = unet_predict(img)`
> * `report = generate_report(mask)`
> * *痛点*：如果我想把 U-Net 换成 ODP-Net，或者想在 predict 前加一个“去噪”步骤，我得去改这一大坨胶水代码。
> 
> 
> * **LangChain (LCEL)**：就像 **Nipype** 或 **Linux 管道**。
> * `pipeline = Loader | Denoise | Segmentor | Reporter`
> * *优势*：**模块化**。我想换模型？拔下来插个新的上去就行。我想加日志？中间插个环节就行。
> 
> 
> 
> 

**LCEL 语法核心**：



符号 `|` 的意思是：**左边的输出，自动成为右边的输入。**


## 💻 2 代码实现 (Notebook 实验)

### 2.1 连接大模型 (The Model)

LangChain 把所有 LLM 都抽象成了 `ChatOpenAI` 类（不要被名字迷惑，只要兼容 OpenAI 协议的都能用）。

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

# 初始化模型 (LangChain 帮你处理了 API 调用细节)
# 注意：model_name 要填你硅基流动的实际模型名
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",  # 你的模型名
    openai_api_key=os.getenv("LLM_API_KEY"),
    openai_api_base=os.getenv("LLM_BASE_URL"),
    temperature=0.7
)

# 测试一下 (直接 invoke)
response = llm.invoke("你好，你是谁？")
print(response.content) 
# 注意：返回的是一个 AIMessage 对象，不是纯字符串


你好呀！我是 DeepSeek-R1，你可以把我当作一个知识小伙伴～🧠

我可以帮你查资料、写文章、解题、翻译、学习、办公、聊天……只要你有需要，我都会尽力帮忙！而且我24小时在线，随时等你来问～

有什么我可以帮你的吗？😊


### Step 2: 模板管理 (The Prompt)

不再用 f-string，而是用 `ChatPromptTemplate`，它支持 System/User 角色分离。

In [3]:
from langchain_core.prompts import ChatPromptTemplate

# 定义模版
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一名资深的法律顾问，擅长处理{case_type}类案件。"),
    ("user", "案情描述：{text}。请简短给出法律建议。")
])

# 看看它渲染出来是什么样
print(prompt_template.invoke({"case_type": "民事", "text": "邻居家的树倒了砸坏了我的车"}))

messages=[SystemMessage(content='你是一名资深的法律顾问，擅长处理民事类案件。', additional_kwargs={}, response_metadata={}), HumanMessage(content='案情描述：邻居家的树倒了砸坏了我的车。请简短给出法律建议。', additional_kwargs={}, response_metadata={})]


### 2.3 组装流水线 (The Chain via LCEL) 🚀

这是 LangChain 的高光时刻。我们将 Prompt、Model 和 OutputParser 串联起来。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 1. 定义解析器 (自动把 AIMessage 对象转成纯字符串)
parser = StrOutputParser()

# 2. 组装链条 (LCEL 语法)
# 逻辑：输入字典 -> 填入 Prompt -> 喂给 LLM -> 提取字符串
chain = prompt_template | llm | parser

# 3. 运行链条
result = chain.invoke({
    "case_type": "侵权", 
    "text": "邻居家的树倒了砸坏了我的车，但他拒绝赔偿。"
})

print("--- AI 建议 ---")
print(result)

--- AI建议 ---

关于邻居树木倒塌致车辆损坏的法律建议：

1. **收集证据**：立即拍摄树木倒伏现场照片、车辆受损照片，记录事发时间及周围环境（如是否有明显风力、雨情等）。若曾就树木安全隐患进行过沟通或收到邻居承诺处理，应保留书面记录或录音。

2. **确认过错归属**：根据《民法典》第125条，树木所有权人或管理人若未尽到维护义务（如未及时修剪、存在明显枯枝断裂风险等），则需承担赔偿责任。需初步判断：树木是否属于年久失修状态？倒伏是否因自然灾害（如台风）导致？若有证据证明邻居存在过错，可主张赔偿。

3. **发起协商**：书面通知邻居，明确要求其承担赔偿责任，并给予合理期限协商。协商不成时，可考虑通过基层调解组织或12348法律援助热线寻求调解。

4. **诉讼途径**：若调解失败，可向人民法院提起侵权责任纠纷诉讼，主张赔偿车辆维修费用及相关损失。需注意收集维修发票、评估报告等证据，并明确诉讼请求金额。

**建议行动**：立即固定证据并发出书面索赔通知，必要时咨询专业律师评估案件可行性。诉讼时效为三年，自权利受损之日起计算。

（注：具体责任认定需结合现场勘查及双方过错程度，建议尽快启动维权程序）


> **👀 观察**：
> 你的代码逻辑变得极其清晰。没有 `json.loads`，没有 `requests.post`，只有业务逻辑的流转。

### 2.4 结构化输出重构 (Structured Output Refactor)

还记得昨天写的 `Pydantic` 解析吗？LangChain 内置了更好的支持。

In [6]:
# Cell 4
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate # 记得导入这个

# 1. 定义数据结构
class LegalAdvice(BaseModel):
    summary: str = Field(description="案情一句话摘要")
    advice: str = Field(description="具体的法律建议")
    confidence: float = Field(description="胜诉概率预估，0-1之间")

# 2. 创建解析器
pydantic_parser = PydanticOutputParser(pydantic_object=LegalAdvice)

# 3. 获取格式指令
format_instructions = pydantic_parser.get_format_instructions()

# 4. 定义 Prompt (挖了两个坑)
prompt_struct = ChatPromptTemplate.from_template(
    """
    你是一名律师。分析以下案情。
    {format_instructions}
    
    案情：{text}
    """
)

# ⭐️ 核心修正：使用 partial 将 format_instructions "预填充" 进去
# 这样新的 prompt_final 就只剩下一个 {text} 坑了
prompt_final = prompt_struct.partial(format_instructions=format_instructions)

# 5. 新的链条 (注意这里用的是 prompt_final)
chain_struct = prompt_final | llm | pydantic_parser

# 6. 运行 (现在只需要传 text 即可)
try:
    result_obj = chain_struct.invoke({"text": "张三借了我5000元不还，只有微信聊天记录。"})
    print("\n--- 结构化结果 ---")
    print(f"摘要: {result_obj.summary}")
    print(f"概率: {result_obj.confidence}")
    print(type(result_obj)) 
except Exception as e:
    print(f"解析失败: {e}")


--- 结构化结果 ---
摘要: 张三承认借款但仅有聊天记录作为证据，需进一步收集证据并评估胜诉概率。
概率: 0.4
<class '__main__.LegalAdvice'>


## 🧠 3 深度理论 & 面试必问 (Deep Dive)

### 🔥 Q1: LangChain 的核心价值是什么？

**面试官想听到的关键词**：**抽象 (Abstraction)**、**编排 (Orchestration)**、**生态 (Ecosystem)**。

**参考回答**：

1. **组件抽象 (Abstraction)**：
* 它将不同的 LLM (OpenAI, DeepSeek, Llama) 统一抽象为 `ChatModel` 接口。
* 就像 **PyTorch 的 Dataset 类**，无论底层是 CSV 还是 JPG，上层训练代码 `DataLoader` 都不用改。这避免了**供应商锁定 (Vendor Lock-in)**。


2. **链式编排 (Chaining / LCEL)**：
* 通过 Unix 管道风格的 `|` 操作符，将 Prompt、Model、Parser、Retriever (检索器) 串联起来，极大地提高了代码的**可读性**和**复用性**。


3. **生态集成 (Integrations)**：
* 它内置了与 向量数据库 (Chroma/Milvus)、搜索引擎 (Google/Bing)、工具 (Calculator/Python) 的接口。开发者不需要自己去写这些复杂的对接代码。



## 🧩 4 今日 LeetCode 练习

LangChain 的核心是 **Chain (链)**。为了致敬这个概念，今天复习链表中最经典的题目。

* **题目**: **[Merge Two Sorted Lists (LC 21)](../../LeetCode%20practice/1-50.ipynb)** - 简单
* **关联**:
* LCEL 的 `Chain` 就像一个单向链表，数据从 Head 流向 Tail。
* 理解指针的移动和节点的连接，有助于理解 LangChain 内部的数据流向。

## ✅ 5 今日任务总结

* [ ] 安装 `langchain` 库。
* [ ] 在 Notebook 中跑通了 `ChatOpenAI` 连接硅基流动。
* [ ] 掌握了 LCEL 写法：`chain = prompt | model | parser`。
* [ ] 使用 LangChain 内置的 `PydanticOutputParser` 实现了结构化输出。
* [ ] **面试准备**：理解 LangChain 的“适配器”本质。

**完成后，请回复“W8D4 完成”，我们将进入 Week 8 的最后一块拼图——让 AI 拥有“记忆” (Memory)！**